In [ ]:
# List of keywords you want to search for. For large scale, replace this with a CSV file and keep kw data under a column.
keywords = ["google analytics 4 bigquery", "gsc bq schema", "how does the bigquery export work"]

In [ ]:
import requests
import json
import time
import os
from datetime import datetime

# API endpoint
url = "https://api.dataforseo.com/v3/serp/google/organic/live/advanced"
# Your authentication credentials
headers = {
    'Authorization': 'Basic write_key_here', # This is where you add your key, leave Basic as it is
    'Content-Type': 'application/json'
}

# Create a results directory if it doesn't exist
results_dir = "serp_results"
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

# Get timestamp for the run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Process each keyword
for keyword in keywords:
    # Create the payload for the current keyword
    payload = json.dumps([{
        "keyword": keyword,
        "location_code": 2840,
        "language_code": "en",
        "device": "mobile",
        "os": "ios",
        "depth": 10,
        "calculate_rectangles": True,
        "group_organic_results": True,
        "load_async_ai_overview":True
    }])

    # Send the request
    response = requests.request("POST", url, headers=headers, data=payload)

    # Print the response status for the current keyword
    print(f"Results for keyword: {keyword}")
    print(f"Status code: {response.status_code}")

    # Save the response to a file
    # Create a safe filename from the keyword
    safe_keyword = keyword.replace(" ", "_").replace("/", "_").replace("\\", "_")
    filename = f"{results_dir}/{timestamp}_{safe_keyword}.json"

    with open(filename, "w") as file:
        # If the response is valid JSON, pretty print it to the file
        try:
            json_response = response.json()
            json.dump(json_response, file, indent=4)
            print(f"Results saved to {filename}")
        except json.JSONDecodeError:
            # If not valid JSON, save the raw text
            file.write(response.text)
            print(f"Non-JSON response saved to {filename}")

    print("-" * 50)

    # Add a short delay to avoid hitting rate limits
    time.sleep(1)

print(f"All results have been saved to the '{results_dir}' directory.")

Results for keyword: google analytics 4 bigquery
Status code: 401
Results saved to serp_results/20250526_033729_google_analytics_4_bigquery.json
--------------------------------------------------
Results for keyword: gsc bq schema
Status code: 401
Results saved to serp_results/20250526_033729_gsc_bq_schema.json
--------------------------------------------------
Results for keyword: how does the bigquery export work
Status code: 401
Results saved to serp_results/20250526_033729_how_does_the_bigquery_export_work.json
--------------------------------------------------
All results have been saved to the 'serp_results' directory.


Now we combine all the files into one:

In [ ]:
import pandas as pd
from pandas import json_normalize
import json
import os
import glob

# Directory where the JSON files are saved
results_dir = "serp_results"

# Create an empty list to store all results
all_results = []

# Get all JSON files in the results directory
json_files = glob.glob(f"{results_dir}/*.json")

# Process each JSON file
for json_file in json_files:
    print(f"Processing file: {json_file}")

    # Extract the keyword from the filename
    filename = os.path.basename(json_file)
    # Assuming filename format: timestamp_keyword.json from the first script
    # We need to remove the timestamp part (YYYYMMDD_HHMMSS_) and the file extension
    parts = filename.split('_')
    # Remove timestamp parts (first two elements) and file extension
    if len(parts) >= 3:  # Make sure we have at least: date_time_keyword.json
        keyword_parts = parts[2:]  # Skip the date and time parts
        keyword = ' '.join([part.replace('.json', '') for part in keyword_parts])
    else:
        # Fallback in case the filename format is different
        print(f"Warning: Unexpected filename format: {filename}")
        keyword = filename.replace('.json', '')

    try:
        # Read the JSON file
        with open(json_file, 'r') as file:
            response_data = json.load(file)

        # Check if the request was successful
        if isinstance(response_data, list):
            # Handle the case when response is a list
            for task_data in response_data:
                if task_data.get("status_code") == 20000:
                    # Extract the organic search results
                    try:
                        # Navigate through the JSON structure to get to the organic results
                        result_items = task_data.get("result", [])
                        for result in result_items:
                            organic_results = result.get("items", [])

                            # Add the current keyword to each result for reference
                            for organic_result in organic_results:
                                organic_result["keyword"] = keyword
                                all_results.append(organic_result)
                    except Exception as e:
                        print(f"Error processing results in {json_file}: {str(e)}")
                else:
                    print(f"Error in {json_file}: {task_data.get('status_message')}")
        elif response_data.get("status_code") == 20000:
            # Handle the case when response is a dictionary
            try:
                # Navigate through the JSON structure to get to the organic results
                tasks = response_data.get("tasks", [])
                for task in tasks:
                    task_result = task.get("result", [])
                    for result in task_result:
                        organic_results = result.get("items", [])

                        # Add the current keyword to each result for reference
                        for organic_result in organic_results:
                            organic_result["keyword"] = keyword
                            all_results.append(organic_result)
            except Exception as e:
                print(f"Error processing results in {json_file}: {str(e)}")
        else:
            print(f"Error in {json_file}: {response_data.get('status_message')}")
    except Exception as e:
        print(f"Error reading or parsing {json_file}: {str(e)}")

# Convert all results to a DataFrame
if all_results:
    print(f"Total results collected: {len(all_results)}")

    # Use json_normalize to flatten nested JSON
    df = json_normalize(all_results)

    # Display the first few rows
    print("Preview of the first few rows:")
    print(df.head())

    # Save to CSV
    output_file = 'dataforseo_combined_results.csv'
    df.to_csv(output_file, index=False)
    print(f"All results saved to '{output_file}'")
else:
    print("No results found in any file.")

Processing file: serp_results/20250526_033729_gsc_bq_schema.json
Error in serp_results/20250526_033729_gsc_bq_schema.json: You are not Authorized to Access this Resource. Your Login Information Here: https://app.dataforseo.com/login .
Processing file: serp_results/20250526_033729_how_does_the_bigquery_export_work.json
Error in serp_results/20250526_033729_how_does_the_bigquery_export_work.json: You are not Authorized to Access this Resource. Your Login Information Here: https://app.dataforseo.com/login .
Processing file: serp_results/20250526_033729_google_analytics_4_bigquery.json
Error in serp_results/20250526_033729_google_analytics_4_bigquery.json: You are not Authorized to Access this Resource. Your Login Information Here: https://app.dataforseo.com/login .
No results found in any file.


## Visualizing SERP Types

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd


# Check if 'type' column exists
if 'type' not in df.columns:
    print("Column 'type' not found. Available columns are:", df.columns.tolist())
else:
    # Count the frequency of each type
    type_counts = df['type'].value_counts().reset_index()
    type_counts.columns = ['Type', 'Count']

    # Calculate percentages
    type_counts['Percentage'] = (type_counts['Count'] / type_counts['Count'].sum() * 100).round(2)

    # Create a horizontal bar chart with Plotly
    fig = go.Figure()

    # Color scale for gradient effect
    colors = px.colors.sequential.Plasma

    # Add the horizontal bar chart with custom styling
    fig.add_trace(go.Bar(
        y=type_counts['Type'],
        x=type_counts['Count'],
        text=type_counts['Count'],
        textposition='outside',
        textfont=dict(
            family='Trebuchet MS, sans-serif',
            size=14,
            color='rgba(50, 50, 50, 0.8)'
        ),
        orientation='h',
        marker=dict(
            color=type_counts['Count'],
            colorscale=colors,
            line=dict(color='rgba(0,0,0,0.3)', width=1.5)
        ),
        hovertemplate='<b>%{y}</b><br>Count: %{x}<br>Percentage: %{customdata:.2f}%<extra></extra>',
        customdata=type_counts['Percentage']
    ))

    # Customize layout
    fig.update_layout(
        title={
            'text': 'Distribution of Result Types',
            'y':0.97,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
                family='Raleway, sans-serif',
                size=28,
                color='rgba(0,0,0,0.85)'
            )
        },
        yaxis_title="",
        xaxis_title="",
        template="plotly_white",
        height=max(500, len(type_counts) * 45),  # Dynamic height based on number of categories
        width=950,
        yaxis={
            'categoryorder':'total ascending',
            'tickfont': dict(
                family='Roboto, sans-serif',
                size=16,
                color='rgba(0,0,0,0.75)'
            ),
            'gridwidth': 0.1,
            'gridcolor': 'rgba(0,0,0,0.05)'
        },
        xaxis={
            'showgrid': True,
            'gridwidth': 0.1,
            'gridcolor': 'rgba(0,0,0,0.05)',
            'zeroline': False,
            'tickfont': dict(
                family='Roboto, sans-serif',
                size=14,
                color='rgba(0,0,0,0.6)'
            )
        },
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Montserrat, sans-serif",
            bordercolor="rgba(0,0,0,0.2)"
        ),
        margin=dict(l=20, r=60, t=80, b=40),
        bargap=0.25,  # Gap between bars
        plot_bgcolor='rgba(250,250,250,0.95)'  # Slight off-white background
    )

    # Add subtle grid lines
    fig.update_yaxes(showgrid=False)

    # Display the interactive chart
    fig.show()

    # Print the distribution summary as a styled table
    fig_table = go.Figure(data=[go.Table(
        header=dict(
            values=['Type', 'Count', 'Percentage (%)'],
            fill_color='rgba(82, 45, 128, 0.8)',  # Deep purple header
            align='center',
            font=dict(
                family='Raleway, sans-serif',
                color='white',
                size=15
            )
        ),
        cells=dict(
            values=[type_counts['Type'], type_counts['Count'], type_counts['Percentage']],
            fill_color=[['rgba(237, 231, 246, 0.5)', 'rgba(248, 245, 255, 0.5)'] * (len(type_counts)//2 + 1)],
            align='center',
            font=dict(
                family='Roboto, sans-serif',
                size=14,
                color=['rgba(0,0,0,0.7)']
            ),
            height=30
        ))
    ])

    fig_table.update_layout(
        title={
            'text': "Distribution Summary",
            'font': dict(
                family='Raleway, sans-serif',
                size=22,
                color='rgba(0,0,0,0.85)'
            )
        },
        height=max(350, len(type_counts) * 30 + 100),
        margin=dict(l=10, r=10, t=50, b=15)
    )

    fig_table.show()

NameError: name 'df' is not defined

In [ ]:
pd.read_csv("/content/dataforseo_combined_results.csv").head(50)

## Pixel Ranking

In [ ]:
# Define above fold threshold (in pixels)
above_fold_threshold = 400

# Create a new column indicating whether the element is above the fold
df['is_above_fold'] = df['rectangle.y'] < above_fold_threshold

# Count how many elements are above/below the fold
above_fold_count = df['is_above_fold'].sum()
below_fold_count = len(df) - above_fold_count

print(f"Total elements: {len(df)}")
print(f"Elements above the fold: {above_fold_count} ({above_fold_count/len(df)*100:.2f}%)")
print(f"Elements below the fold: {below_fold_count} ({below_fold_count/len(df)*100:.2f}%)")

Total elements: 40
Elements above the fold: 5 (12.50%)
Elements below the fold: 35 (87.50%)


Doing the same but for each keyword:

In [ ]:
# Define above fold threshold (in pixels)
above_fold_threshold = 400

# Create a new column indicating whether the element is above the fold
df['is_above_fold'] = df['rectangle.y'] < above_fold_threshold

# Group by keyword and calculate counts for each
keyword_counts = df.groupby('keyword').apply(
    lambda group: {
        'total': len(group),
        'above_fold': group['is_above_fold'].sum(),
        'below_fold': len(group) - group['is_above_fold'].sum()
    },         include_groups=False
).to_dict()

# Print results for each keyword
for keyword, counts in keyword_counts.items():
    total = counts['total']
    above = counts['above_fold']
    below = counts['below_fold']

    print(f"\nKeyword: {keyword}")
    print(f"Total elements: {total}")
    print(f"Elements above the fold: {above} ({above/total*100:.2f}%)")
    print(f"Elements below the fold: {below} ({below/total*100:.2f}%)")

# Overall totals (optional)
print("\nOVERALL TOTALS:")
print(f"Total elements: {len(df)}")
print(f"Elements above the fold: {df['is_above_fold'].sum()} ({df['is_above_fold'].sum()/len(df)*100:.2f}%)")
print(f"Elements below the fold: {len(df) - df['is_above_fold'].sum()} ({(len(df) - df['is_above_fold'].sum())/len(df)*100:.2f}%)")


Keyword: google analytics 4 bigquery
Total elements: 13
Elements above the fold: 2 (15.38%)
Elements below the fold: 11 (84.62%)

Keyword: gsc bq schema
Total elements: 13
Elements above the fold: 2 (15.38%)
Elements below the fold: 11 (84.62%)

Keyword: how does the bigquery export work
Total elements: 14
Elements above the fold: 1 (7.14%)
Elements below the fold: 13 (92.86%)

OVERALL TOTALS:
Total elements: 40
Elements above the fold: 5 (12.50%)
Elements below the fold: 35 (87.50%)


In [ ]:
df.head(50)